In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_data = pd.read_csv('results/clusters.tsv', sep='\t')

In [3]:
uri_to_cluster = dict()
clusters = []
is_main_label = dict()
uri_to_label = dict()

for _, r in raw_data.iterrows():
    s = r['a']
    o = r['b']

    uri_to_label[s] = r['label']

    if 'preferential' in r['prop']:
        is_main_label[o] = True
    elif o not in is_main_label:
        is_main_label[o] = False

    cluster_no = uri_to_cluster.get(s, None)
    if cluster_no is None:
        cluster_no = uri_to_cluster.get(o, None)

    if cluster_no is None:
        uri_to_cluster[s] = len(clusters)
        uri_to_cluster[o] = len(clusters)
        clusters.append({s, o})
    else:
        uri_to_cluster[s] = cluster_no
        uri_to_cluster[o] = cluster_no
        clusters[cluster_no].add(s)
        clusters[cluster_no].add(o)

In [4]:
pd.to_pickle((
    clusters, uri_to_cluster, is_main_label, uri_to_label
), 'temp/cluster_data.pkl.bz2')

In [5]:
topics = pd.read_pickle('temp/topics.pkl.bz2')

In [6]:
clusters_sim = []
clusters_seq = []

for cluster in clusters:
    seq = list(cluster)
    clusters_seq.append(seq)

    arr = np.vstack(topics['encoding'].loc[cluster])
    clusters_sim.append(arr.dot(arr.T))

In [7]:
pd.to_pickle((
    clusters_seq, clusters_sim, is_main_label, uri_to_label
), 'temp/clusters.pkl.bz2')